# YAML Joiner

## Import module



In [1]:
# Import the modules

import os

In [2]:
def import_file(file):
    with open(file) as f:
        lines = f.readlines()
        
    dependencies = []
    pip = []
    write_dep = False
    write_pip = False
    
    for line in lines:
        if write_dep:
            dependencies.append(line)
        if write_pip:
            pip.append(line)
            
        if "dependencies" in line:
            write_dep = True
        
        if "pip:" in line:
            write_pip = True
            write_dep = False
            
        
        if "prefix:" in line:
            break
    
    dependencies = dependencies[:-1]
    pip = pip[:-1]
    return dependencies, pip

In [3]:
slave_dep, slave_pip = import_file("src/PDFv1.yaml")
master_dep, master_pip = import_file("src/proto_bal.yaml")

In [4]:
def add_master(masters, slaves):
    
    output = []
    for slave in slaves:
        
        slave_name = slave[:slave.index("=")]
        counter = 0
        
        for master in masters:
            if slave_name in master:
                counter +=1
                break
        
        if counter == 0:
            output.append(slave)
            
    return output

In [5]:
add_master(master_dep, slave_dep)

[]

In [6]:
add_master(master_pip, slave_pip)

['  - argcomplete==1.8.2\n',
 '  - beautifulsoup4==4.5.3\n',
 '  - distribute==0.7.3\n',
 '  - docx2txt==0.6\n',
 '  - ebooklib==0.15\n',
 '  - lxml==4.1.0\n',
 '  - olefile==0.44\n',
 '  - pdfminer==20140328\n',
 '  - pillow==4.3.0\n',
 '  - pocketsphinx==0.1.3\n',
 '  - pypdf2==1.26.0\n',
 '  - python-pptx==0.6.5\n',
 '  - slate==0.3\n',
 '  - speechrecognition==3.6.3\n',
 '  - textract==1.6.1\n',
 '  - xlrd==1.0.0\n',
 '  - xlsxwriter==1.0.2\n']

In [7]:
slave_pip

['  - argcomplete==1.8.2\n',
 '  - backports-abc==0.5\n',
 '  - backports.shutil-get-terminal-size==1.0.0\n',
 '  - backports.ssl-match-hostname==3.5.0.1\n',
 '  - beautifulsoup4==4.5.3\n',
 '  - chardet==2.3.0\n',
 '  - distribute==0.7.3\n',
 '  - docx2txt==0.6\n',
 '  - ebooklib==0.15\n',
 '  - ipython-genutils==0.2.0\n',
 '  - jupyter-client==5.1.0\n',
 '  - jupyter-console==5.2.0\n',
 '  - jupyter-core==4.3.0\n',
 '  - lxml==4.1.0\n',
 '  - olefile==0.44\n',
 '  - pdfminer==20140328\n',
 '  - pillow==4.3.0\n',
 '  - pocketsphinx==0.1.3\n',
 '  - prompt-toolkit==1.0.15\n',
 '  - pypdf2==1.26.0\n',
 '  - python-pptx==0.6.5\n',
 '  - slate==0.3\n',
 '  - speechrecognition==3.6.3\n',
 '  - textract==1.6.1\n',
 '  - win-unicode-console==0.5\n',
 '  - xlrd==1.0.0\n',
 '  - xlsxwriter==1.0.2\n']